In [ ]:
import requests
import time

In [2]:
TCG_API_KEY = '831a5c03-fe5a-4632-b875-4a5144a170e3'

In [ ]:
def get_cards(page, page_size=150, max_retries=3):
    url = 'https://api.pokemontcg.io/v2/cards'
    headers = {
        'X-Api-Key': TCG_API_KEY
    }
    params = {
        'page': page, 
        'pageSize': page_size
    }
    
    for attempt in range(max_retries + 1):

        try:
            response = requests.get(url, headers=headers, params=params, timeout=120)
            response.raise_for_status()
            data = response.json()
            
            return data   
        except requests.exceptions.RequestException as e:
            print(f'Error fetching page {page}: {e}')

            if attempt == max_retries:
                print(f'Failed to fetch page {page} after {max_retries} retries')
                return page
            else:
                wait_time = 2 ** (attempt + 1)
                time.sleep(wait_time)

In [8]:
def get_cards(page=1):
    binder = []
    missed_pages = []

    while True:
        info = fetch_cards(page)
        
        if isinstance(info, dict):
            count = info.get('count', 0)
            if (count > 0): 
                cards = info.get('data', None)
                binder.extend(cards)
            else:
                break 
        else:
            missed_pages.append(page)
        page += 1
    
    while missed_pages:
        page = missed_pages.pop()
        print(f'Second attempt for page {page}.')
        info = fetch_cards(page)
        
        if isinstance(info, dict):  
            count = info.get('count', 0)
            if (count > 0):
                cards = info.get('data', None)
                binder.extend(cards)
        else:
            print(f'Second attempt failed for page {page}.')
            
    return binder

In [ ]:
cards = get_cards()

In [10]:
len(cards)

19500

In [ ]:
import os
from pathlib import Path
import json

In [ ]:
with open('cards.json', 'w') as file:
    json.dump(cards, file)

In [ ]:
with open("cards.json", "r") as file:
    cards = json.load(file)

In [45]:
out_dir = 'cards'
os.makedirs(out_dir, exist_ok=True)

In [46]:
for card in cards:
    id = card['id']
    file_name = f"{id}.json"
    path = os.path.join(out_dir, file_name)
    with open(path, 'w') as file:
        json.dump(card, file)

In [ ]:
from snowflake.snowpark import Session
from datetime import datetime

In [ ]:
connection = {
    "account": "NAEWJSH-MN92912",
    "user": "VOTUAN3719",
    "password": "Dougotrice1036",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "POKEMON",
    "schema": "PUBLIC",
}
session = Session.builder.configs(connection).create()

In [69]:
session

In [70]:
session.sql("CREATE OR REPLACE STAGE cards").collect()

[Row(status='Stage area CARDS successfully created.')]

In [ ]:
folder = Path("cards").resolve()
date = datetime.now().strftime("%Y-%m-%d")
session.file.put(
    f"file://{folder}/*.json",   
    f"@cards/{date}",
    auto_compress=False,
    overwrite=True,
)

In [74]:
session.close()